# 2. The BlueiceExtendedModel: Fitting and Confidence Intervals
In the previous tutorial we learned about the `BlueiceExtendedModel` and its rate and shape parameters. Now, we'll make use of this knowledge to fit the model to data and compute a confidence interval for the WIMP rate parameter.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from alea import BlueiceExtendedModel

In [2]:
# Just some plotting settings
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['figure.figsize'] = [4, 3]
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 9

## 2.1 Initializing the model & generating data
This is the same as in the previous notebook.

In [3]:
# initialize
config_path = 'unbinned_wimp_statistical_model_simple.yaml'
model = BlueiceExtendedModel.from_config(config_path)

# generate and assign data
data = model.generate_data()
model.data = data

Computing/loading models on one core: 100%|██████████| 5/5 [00:00<00:00, 367.10it/s]


## 2.2 Fitting the data

Now that the data is set, we can perform the unconditional fit as in the first example notebook:

In [4]:
best_fit, max_ll = model.fit()
best_fit

{'wimp_mass': 50.0,
 'livetime': 2.0,
 'wimp_rate_multiplier': 1.2627170044682268,
 'er_rate_multiplier': 0.9695221032538812,
 'er_band_shift': 0.03394646399597126}

Let's check the expectation values under the best-fit parameters:

In [5]:
model.get_expectation_values(**best_fit)

{'er': 387.8088413015525, 'wimp': 25.25434008936454}

Since we use [iminuit](https://iminuit.readthedocs.io/en/stable/index.html) as a backend for minimizing the likelihood, we can have a look at the very detailed iminuit output, which provides much more information about the fit:

In [6]:
model.minuit_object

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 3035                       │              Nfcn = 66               │
│ EDM = 7.67e-07 (Goal: 0.0001)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────────────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name                 │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────────────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ wimp_mass            │   50.0    │    0.5    │            │            │         │         │  yes  │
│ 1 │ livetime             │   2.00    │   0.02    │            │            │         │         │  yes  │
│ 2 │ wimp_rate_multiplier │   1.26    │   0.32    │            │            │    0    │         │       │
│ 3 │ er_rate_multiplier   │   0.97    │   0.05    │            │            │    0    │         │       │
│ 4 │ er_band_shift        │   0.03    │   0.07    │            │            │   -2    │    2    │       │
└───┴──────────────────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                      │            wimp_mass             livetime wimp_rate_multiplier   er_rate_multiplier        er_band_shift │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│            wimp_mass │                    0                    0                  0.0               0.0000                0.000 │
│             livetime │                    0                    0                  0.0               0.0000                0.000 │
│ wimp_rate_multiplier │                  0.0                  0.0                  0.1              -0.0018                0.002 │
│   er_rate_multiplier │               0.0000               0.0000              -0.0018              0.00241              -0.0001 │
│        er_band_shift │                0.000                0.000                0.002              -0.0001              0.00465 │
└──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

You can see that the parameters, which are not fittable are fixed in the fit to their nominal values. For comparison let's have a look at our parameter definition again:

In [7]:
print(model.parameters)

                      nominal_value  fittable  ptype  uncertainty relative_uncertainty    blueice_anchors fit_limits parameter_interval_bounds  fit_guess                                               description
wimp_mass                      50.0     False   None          NaN                 None               None       None                      None        NaN                                      WIMP mass in GeV/c^2
livetime                        2.0     False   None          NaN                 None               None       None                      None        NaN                                         Livetime in years
wimp_rate_multiplier            1.0      True   rate          NaN                 None               None  [0, None]                   [0, 50]        NaN                                                      None
er_rate_multiplier              1.0      True   rate          0.2                 True               None  [0, None]                      None        1.

To perform a conditional fit, i.e. fix a parameter to a certain value, we can simply parse this value to the `fit`` method with the value of the parameter we want to fix. For example we could perform the background-only fit by fixing the `wimp_rate_multiplier` to 0.

In [8]:
best_fit_c, max_ll_c = model.fit(wimp_rate_multiplier=0.)
best_fit_c

{'wimp_mass': 50.0,
 'livetime': 2.0,
 'wimp_rate_multiplier': 0.0,
 'er_rate_multiplier': 1.029804318674445,
 'er_band_shift': -0.14457157553431318}

In [9]:
model.minuit_object

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 3082                       │              Nfcn = 46               │
│ EDM = 2.71e-08 (Goal: 0.0001)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────────────────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name                 │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────────────────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ wimp_mass            │   50.0    │    0.5    │            │            │         │         │  yes  │
│ 1 │ livetime             │   2.00    │   0.02    │            │            │         │         │  yes  │
│ 2 │ wimp_rate_multiplier │    0.0    │    0.1    │            │            │    0    │         │  yes  │
│ 3 │ er_rate_multiplier   │   1.03    │   0.05    │            │            │    0    │         │       │
│ 4 │ er_band_shift        │   -0.14   │   0.07    │            │            │   -2    │    2    │       │
└───┴──────────────────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                      │            wimp_mass             livetime wimp_rate_multiplier   er_rate_multiplier        er_band_shift │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│            wimp_mass │                    0                    0                    0               0.0000                0.000 │
│             livetime │                    0                    0                    0               0.0000                0.000 │
│ wimp_rate_multiplier │                    0                    0                    0               0.0000                0.000 │
│   er_rate_multiplier │               0.0000               0.0000               0.0000              0.00245               0.0000 │
│        er_band_shift │                0.000                0.000                0.000               0.0000              0.00482 │
└──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [10]:
model.get_expectation_values(**best_fit_c)

{'er': 411.9217320132483, 'wimp': 0.0}

You can nicely see that in the constrained fit, the missing WIMP signal component is absorbed by a higher ER rate and a lower ER shift parameter (shifting the ER band down in cs2 towards the WIMP signals in our toy data).

## 2.3 The ancillary likelihood term

In [ ]:
# TODO: Illustrate the impact of the ancillary term by putting the er rate multiplier somewhere ridiculous

## 2.4 Constructing confidence intervals

In [11]:
model.confidence_interval('wimp_rate_multiplier')

(0.8035429129786587, 1.8475071911094776)

In [ ]:
# TODO: Illustrate the asymptotic construction of the CI